# Logistic regression model
Goal is to make a model that can predict when spot 02 will be available

1st model, assume each time point is independent



## 0 imports

In [ ]:
import os
if 'models' == os.getcwd().split('/')[-1]: os.chdir('..')
if 'ev_charging' == os.getcwd().split('/')[-1]: print('in the right place!')
else: os.chdir('/Users/varunvenkatesh/Documents/Github/ev_charging')
os.getcwd()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.data_preprocessing import datetime_processing, userinput_processing, holiday_processing

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, precision_score, recall_score, accuracy_score


## 1 Data and Cleaning

In [ ]:
# load data

In [ ]:
df_of = pd.read_parquet('data/ACN-API/office001/').reset_index(drop=True)
df_of = datetime_processing(df_of)
df_of = userinput_processing(df_of)
df_of = holiday_processing(df_of)


In [ ]:
df_of.head()

In [ ]:
df_of.info()


## 2 plotting and eval

In [ ]:
def make_classification_plot(cm):
    disp = ConfusionMatrixDisplay(cm)
    disp = disp.plot(include_values=True, cmap='viridis', ax=None, xticks_rotation='horizontal')
    plt.grid(False)
    plt.show()

In [ ]:
def get_results(y_test, prediction):
    cm = confusion_matrix(y_test,prediction)
    make_classification_plot(cm)
    
    results = {'tpr': cm[1, 1]/np.sum(cm[1]),
               'fpr': cm[0,1]/np.sum(cm[0]),
               'accuracy': accuracy_score(y_test, prediction),
               'precision': precision_score(y_test, prediction),
               'recall': recall_score(y_test, prediction),
        'f1':f1_score(y_test,prediction)}
    return results
    

In [ ]:
results = {}

# 3 Make X and y values

In [ ]:
tmp = df_of.copy().set_index('connectionTime')
tmp = tmp[tmp['spaceID'] == '02'].sort_index()

In [ ]:
# TODO: make better variable names ie start_ -> session_start_time
y = pd.DataFrame(index=pd.date_range('2019-03-25','2021-09-12', inclusive='both', freq='h', tz=0),columns=['is_available','sessionID'])
y['is_available'] = 1
for i in range(len(tmp)):
    start_ = tmp.index[i]
    end_ = tmp.loc[start_,'disconnectTime'] 
    session_ = tmp.loc[start_,'sessionID']
    # print(start_,'\t', end_,'\t', session_)
    y.loc[start_:end_,['is_available','sessionID']] = 0, session_

In [ ]:
y = y['is_available']

In [ ]:
X = pd.DataFrame(index=pd.date_range('2019-03-25','2021-09-12', inclusive='both', freq='h', tz=0),columns=['dow','hour','month'])
# X['dow'] = X.index.dt.hour
X['dow'] = X.index.dayofweek
X['hour'] = X.index.hour
X['month'] = X.index.month
X['connectionTime'] = X.index
X = holiday_processing(X).drop(columns=['connectionTime'])
X.head()

In [ ]:
# print(f'charger spot #2 is available {np.round(y.is_available.mean()*100,3)}% of the time')
print(f'charger spot #2 is available {np.round(y.mean()*100,3)}% of the time')

In [ ]:
X.shape[0] == y.shape[0]

# 4 model time

## using a stratified train test split as often as possible
https://www.investopedia.com/terms/stratified_random_sampling.asp

In [ ]:
# Create hold out test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, stratify=y)
print(f'the training data has an average availability of {np.round(y_train.mean()*100,3)}%')


## 4.1 Baselines

### 4.10 Guess always available (baseline)

In [ ]:
print('guess always available and never available')
results['always_available'] = get_results(y_test, [1] * len(y_test))


### 4.11 Guess never available (baseline)

In [ ]:
print('guess never available') 
results['never_available'] = get_results(y_test, [0] * len(y_test))

## 4.2 Sklearn logistic regression models

## 4.21 L2 penalty

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline


model = LogisticRegression(penalty='l2')
pipe = Pipeline([
    ('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first')),
    ('lr', model),
])

skf = StratifiedKFold(n_splits=3)
cv_results = cross_validate(pipe, X_train, y_train, cv=skf)
test_score = cv_results["test_score"]
print(
    f"The average accuracy is {test_score.mean():.3f} ± {test_score.std():.3f}"
)
pipe.fit(X_train, y_train)
pred = pipe.predict(X_test)
print(f'test accuracy is {pipe.score(X_test, y_test)}')

results['sk_log_l2_skf'] = get_results(y_test, pred)


In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=skf)

In [ ]:
# standard error for 

In [ ]:
# https://stackoverflow.com/questions/22381497/python-scikit-learn-linear-model-parameter-standard-error
N = len(X_train)
p = len(pipe[0].get_feature_names_out()) + 1  # plus one because LinearRegression adds an intercept term
print('X shape :', N,p)
X_with_intercept = np.empty(shape=(N, p), dtype='float')
X_with_intercept[:, 0] = 1
X_with_intercept[:, 1:p] = pipe[0].transform(X_train)# ohe.transform(X_train)
beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ y_train.values
coefs_intercept = np.concatenate((pipe[1].intercept_, pipe[1].coef_.reshape(-1)))
col_names = ['intercept'] + list(pipe[:-1].get_feature_names_out())
pd.DataFrame(data=np.concatenate([[coefs_intercept], [beta_hat]]),
             columns=['intercept'] + list(pipe[:-1].get_feature_names_out())).T.reset_index().rename(columns={0:'estimation',1:'standard_error','index':'coefficient'})

In [ ]:
# https://stackoverflow.com/questions/22381497/python-scikit-learn-linear-model-parameter-standard-error
X_with_intercept = np.empty(shape=(N, p), dtype='float')
X_with_intercept[:, 0] = 1
X_with_intercept[:, 1:p] = pipe[0].transform(X_train)# ohe.transform(X_train)
X_with_intercept
beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ y_train.values
print(beta_hat)

In [ ]:
pipe[:-1].get_feature_names_out()
pipe[1].coef_
pd.DataFrame(data=np.concatenate([[coefs_intercept], [beta_hat]]),
             columns=['intercept'] + list(pipe[:-1].get_feature_names_out())).T.reset_index().rename(columns={0:'estimation',1:'standard_error','index':'coefficient'})

## 4.22 L1 penalty

In [ ]:
model = LogisticRegression(penalty='l1', solver='liblinear')
pipe = Pipeline([
    ('ohe', OneHotEncoder()),
    ('lrl1', model),
])

skf = StratifiedKFold(n_splits=3)
cv_results = cross_validate(pipe, X_train, y_train, cv=skf)
test_score = cv_results["test_score"]
print(
    f"The average accuracy is {test_score.mean():.3f} ± {test_score.std():.3f}"
)
pipe.fit(X_train, y_train)
pred = pipe.predict(X_test)
print(f'test accuracy is {pipe.score(X_test, y_test)}')

results['sk_log_reg_l1_skf'] = get_results(y_test, pred)


In [ ]:
print(pipe[-1].coef_.shape)
pipe[-1].n_features_in_
pipe[-1].intercept_
pipe[-1].classes_
pd.DataFrame(pipe[-1].coef_, columns=pipe[:-1].get_feature_names_out()).T
# ?pipe[-1]
# pipe[:-1].get_feature_names_out()

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix
# A sample toy binary classification dataset
# X, y = datasets.make_classification(n_classes=2, random_state=0)
# svm = LinearSVC(dual="auto", random_state=0)
lr = LogisticRegression(penalty="l2")
def confusion_matrix_scorer(clf, X, y):
     y_pred = clf.predict(X)
     cm = confusion_matrix(y, y_pred)
     return {'tn': cm[0, 0], 'fp': cm[0, 1],
             'fn': cm[1, 0], 'tp': cm[1, 1]}
cv_results = cross_validate(lr, X_train, y_train, cv=5,
                            scoring=confusion_matrix_scorer)
# Getting the test set true positive scores
print(cv_results['test_tp'])
# Getting the test set false negative scores
print(cv_results['test_fn'])

## 4.23 elastic net penalty

# TO DO: use gridsearch to find best l1 ratio

In [ ]:
model = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.1)
pipe = Pipeline([
    ('ohe', OneHotEncoder()),
    ('lrl1', model),
])

skf = StratifiedKFold(n_splits=3)
cv_results = cross_validate(pipe, X_train, y_train, cv=skf)
test_score = cv_results["test_score"]
print(
    f"The average accuracy is {test_score.mean():.3f} ± {test_score.std():.3f}"
)
pipe.fit(X_train, y_train)
pred = pipe.predict(X_test)
print(f'test accuracy is {pipe.score(X_test, y_test)}')

results['sk_log_elasticnet_skf'] = get_results(y_test, pred)

## 4.3 Sklearn Random Forrest Classifier

In [ ]:
# tree models
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
pipe = Pipeline([
    ('ohe', OneHotEncoder()),
    ('rf', model),
])

skf = StratifiedKFold(n_splits=3)
cv_results = cross_validate(pipe, X_train, y_train, cv=skf)
test_score = cv_results["test_score"]
print(
    f"The average accuracy is {test_score.mean():.3f} ± {test_score.std():.3f}"
)
pipe.fit(X_train, y_train)
pred = pipe.predict(X_test)
print(f'test accuracy is {pipe.score(X_test, y_test)}')

results['sk_rf_classification_skf'] = get_results(y_test, pred)


# 4.4 stats model version

In [ ]:
! pip install statsmodels

In [ ]:
import statsmodels.api as sm 
ohe = OneHotEncoder(sparse_output=False, drop='first')
ohe.fit(X[['dow']])
ohe.transform(X[['dow']])
pd.DataFrame(ohe.transform(X[['dow']]), columns=ohe.get_feature_names_out(), index=X.index)

In [ ]:
ohe = OneHotEncoder(sparse_output=False, drop='first')
ohe.fit(X[['dow']])
ohe.transform(X[['dow']])
ohe.get_feature_names_out()
X_train, X_test, y_train, y_test = train_test_split(
    pd.DataFrame(ohe.transform(X[['dow']]), columns=ohe.get_feature_names_out(), index=X.index), 
    y, 
    test_size=.2, 
    stratify=y)
log_reg = sm.Logit(y_train, X_train).fit() 

In [ ]:
ohe = OneHotEncoder(sparse_output=False)
ohe.fit(X[['dow']])
ohe.transform(X[['dow']])
ohe.get_feature_names_out()
X_train, X_test, y_train, y_test = train_test_split(
    pd.DataFrame(ohe.transform(X[['dow']]), columns=ohe.get_feature_names_out(), index=X.index),#.drop(columns=ohe.get_feature_names_out()[0:3]), 
    y, 
    test_size=.2, 
    stratify=y)
log_reg = sm.Logit(y_train, X_train).fit() 
print(log_reg.summary())

In [ ]:
print(log_reg.summary()) 

In [ ]:
y_hat = log_reg.predict(X_test)
prediction = list(map(round, y_hat))
# confusion matrix 
cm = confusion_matrix(y_test, prediction)  
print ("Confusion Matrix : \n", cm)  
  
# accuracy score of the model 
print('Test accuracy = ', accuracy_score(y_test, prediction))

In [ ]:
results['sm_log_reg_dow_skf'] = get_results(y_test, prediction)

In [ ]:
ohe = OneHotEncoder(sparse_output=False, drop='first')
ohe.fit(X[['hour']])
ohe.transform(X[['hour']])
X_train, X_test, y_train, y_test = train_test_split(
    pd.DataFrame(ohe.transform(X[['hour']]), columns=ohe.get_feature_names_out(), index=X.index), 
    y, 
    test_size=.2, 
    stratify=y)
# X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(ohe.transform(X[['hour']]), columns=ohe.get_feature_names_out(), index=X.index), y['is_available'])
log_reg = sm.Logit(y_train, X_train).fit() 

In [ ]:
print(log_reg.summary()) 

In [ ]:
yhat = log_reg.predict(X_test) 
prediction = list(map(round, yhat))
results['sm_hour_sfk'] = get_results(y_test, prediction)

In [ ]:
X_train

In [ ]:
# not weekend, the log odds is 1.782
# for the weekend, the log odds increases 2.3780, to 4.1606
y_train[X_train.hour_18==True]
# probability of ytrain
print('for not 6 PM...')
p_success = y_train[X_train.hour_18==False].mean()
p_failure = 1- p_success
odds = p_success/p_failure
log_odds = np.log(odds)
print(f'prob of success {p_success}\nprob of failure {p_failure}\nodds of success {odds}\nlog odds of success {log_odds}')
p_not6pm = p_success
print('\n\nfor 6 PM,')
p_success = y_train[X_train.hour_18==True].mean()
p_failure = 1- p_success
odds = p_success/p_failure
log_odds = np.log(odds)
print(f'prob of success {p_success}\nprob of failure {p_failure}\nodds of success {odds}\nlog odds of success {log_odds}')
print(f'\n\nthe probaility of availability changes from {p_success} to {p_not6pm}, a difference of {np.round((p_success-p_not6pm)/p_not6pm*100,3)}%')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, prediction))

In [ ]:
ohe = OneHotEncoder(sparse_output=False, drop='first')
ohe.fit(X[['month','hour']])
ohe.transform(X[['month','hour']])
X_train, X_test, y_train, y_test = train_test_split(
    pd.DataFrame(ohe.transform(X[['month','hour']]), columns=ohe.get_feature_names_out(), index=X.index), 
    y, 
    test_size=.2, 
    stratify=y)
# X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(ohe.transform(X[['hour']]), columns=ohe.get_feature_names_out(), index=X.index), y['is_available'])
log_reg = sm.Logit(y_train, X_train).fit() 

In [ ]:
print(log_reg.summary())

In [ ]:
yhat = log_reg.predict(X_test) 
prediction = list(map(round, yhat))
results['sm_month_hour_skf'] = get_results(y_test, prediction)


In [ ]:
ohe = OneHotEncoder(sparse_output=False, drop='first')
ohe.fit(X)
ohe.transform(X)
ohe.get_feature_names_out()[0]

X_train, X_test, y_train, y_test = train_test_split(
    pd.DataFrame(ohe.transform(X), columns=ohe.get_feature_names_out(), index=X.index), #.drop(columns=ohe.get_feature_names_out()[0]), 
    y, 
    test_size=.2, 
    stratify=y)
# X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(ohe.transform(X[['hour']]), columns=ohe.get_feature_names_out(), index=X.index), y['is_available'])
log_reg = sm.Logit(y_train, X_train).fit() 
print(log_reg.summary())

In [ ]:
yhat = log_reg.predict(X_test) 
prediction = list(map(round, yhat))
results['sm_month_hour_dow_skf'] = get_results(y_test, prediction)

In [ ]:
print('guess always available and never available')
results['always_available'] =  get_results(y_test, [1]*len(y_test))
results['never_available'] =  get_results(y_test, [0]*len(y_test))


In [ ]:
ohe = OneHotEncoder(sparse_output=False, drop='first')
ohe.fit(X[['month']])
ohe.transform(X[['month']])
X_train, X_test, y_train, y_test = train_test_split(
    pd.DataFrame(ohe.transform(X[['month']]), columns=ohe.get_feature_names_out(), index=X.index), 
    y, 
    test_size=.2, 
    stratify=y)
log_reg = sm.Logit(y_train, X_train).fit() 

In [ ]:
print(log_reg.summary())

In [ ]:
yhat = log_reg.predict(X_test) 
prediction = list(map(round, yhat))
yhat

In [ ]:
print(classification_report(y_test, prediction))
confusion_matrix(y_test, prediction)


In [ ]:
from sklearn.metrics import  f1_score, precision_score, recall_score
print(f'Precision: out of all the times the model predicted the charger would be available, {np.round(precision_score(y_test, prediction)*100,3)}% of the time it actually was available')
print(f'Recall: out of all the times the charger was available, the model predicted the outcome correctly for {np.round(recall_score(y_test, prediction)*100,3)}% of those times')
print(f'F1 score: the precision recall balance of the model was {np.round(f1_score(y_test,prediction),3)}')
print(f'support available:   {np.sum(y_test==1)}')
print(f'support unavailable: {np.sum(y_test==0)}')


In [ ]:
pd.DataFrame(results).T

# Interpreting model coefficients

In [ ]:
# probability of ytrain
p_success = y_train.mean()
p_failure = 1- p_success
odds = p_success/p_failure
log_odds = np.log(odds)
print(f'prob of success {p_success}\nprob of failure {p_failure}\nodds of success {odds}\nlog odds of success {log_odds}')

In [ ]:
dummy_1 = pd.DataFrame([1]*len(y_train), columns=['intercept'], index=y_train.index)
int_model = sm.Logit(y_train, dummy_1).fit()

In [ ]:
print(int_model.summary())

In [ ]:
print(f'the model intercept term is {int_model.params.iloc[0]} which is the same as the log odds of the training set {log_odds}')

# new model

In [ ]:
X['is_weekend'] = X['dow'].isin([5,6])
X

In [ ]:
ohe = OneHotEncoder(sparse_output=False, drop='first')
ohe.fit(X[['is_weekend']])
ohe.transform(X[['is_weekend']])
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(ohe.transform(X[['is_weekend']]), columns=ohe.get_feature_names_out(), index=X.index), y)
log_reg = sm.Logit(y_train, sm.add_constant(X_train)).fit() 
print(log_reg.summary())

$logit(p/1-p) = \beta_0 + \beta_1*is_weekend$

In [ ]:
# not weekend, the log odds is 1.782
# for the weekend, the log odds increases 2.3780, to 4.1606
y_train[X_train.is_weekend_True==True]
# probability of ytrain
print('for weekdays...')
p_success = y_train[X_train.is_weekend_True==False].mean()
p_failure = 1- p_success
odds = p_success/p_failure
log_odds = np.log(odds)
print(f'prob of success {p_success}\nprob of failure {p_failure}\nodds of success {odds}\nlog odds of success {log_odds}')
p_notweekend=p_success
print('\n\nfor weekends,')
p_success = y_train[X_train.is_weekend_True==True].mean()
p_failure = 1- p_success
odds = p_success/p_failure
log_odds = np.log(odds)
print(f'prob of success {p_success}\nprob of failure {p_failure}\nodds of success {odds}\nlog odds of success {log_odds}')
print(f'\n\nthe probaility of availability changes from {p_success} to {p_notweekend}, a difference of {np.round((p_success-p_notweekend)/p_notweekend*100,3)}%')

In [ ]:
?log_reg
lowt = log_reg.params.iloc[0] + log_reg.params.iloc[1]
lowf = log_reg.params.iloc[0]
prwt = np.exp(lowt)/(1+np.exp(lowt))
prwf = np.exp(lowf)/(1+np.exp(lowf))
print(f'probability of weekend availability {prwt}\nprobability of weekday availability {prwf}')

In [ ]:
# weekend add 2.4 to the log odds of availability
# p  exp(beta X)/ 1+ exp(beta X)
# the weekend makes spot availability 2.4 times more likely to occur
# the odds of is availability change by exp(2.49) time for c unit increase in x
np.exp(2.493)

In [ ]:
print(f'the change in log odds of the day being a weekend is {2.493}')
print(f'that means that the change in odds is {np.exp(2.493)}')

In [ ]:
is_weekend = 0
P_non_weekend = np.exp(1.7667 + 2.4931 * is_weekend)/(1+np.exp(1.7667 + 2.4931 * is_weekend))
is_weekend = 1
P_weekend = np.exp(1.7667 + 2.4931 * is_weekend)/(1+np.exp(1.7667 + 2.4931 * is_weekend))
print(np.round(P_non_weekend,3), np.round(P_weekend, 3))


In [ ]:
np.exp(5.2057)/(1+np.exp(5.2057))